In [2]:
# CHANGE THESE NUMBERS ONLY

# anchors categorized as a regular grid,
# could modify the code to use the same approach for a grid that isnt regular

x_spacing = 2
y_spacing = 3

no_anchors_x = 4
no_anchors_y = 5

# height is the height of the cone, or the depth of the anchor.
# Generally refering to the bonded length, since for a the unbonded length it may be more appropriate
# to take the volume of a cylinder over the area of the cone being pulled out.

height = 5

In [3]:
# CELL RETURNS THE VOLUME OF THE CONE GROUP

# import shapely
from shapely.geometry import Point
from shapely.ops import unary_union

# create a function that calculates the area of a group of joined circles
def area_circle_group(x_spacing, y_spacing, no_anchors_x, no_anchors_y, diameter):
    circles=[]
    # loop through grid of x and y coordinates
    for x in range(0,no_anchors_x):
        for y in range(0, no_anchors_y):
            # for each x and y coordinate add a circle
            # note circle is actually represented as a 64 sided polygon
            circles.append(Point(x*x_spacing, y*y_spacing).buffer(diameter))

    return unary_union(circles).area

# create a function that integrates by calculating the area of a slice through the group anchors
def volume_group_anchors(x_spacing, y_spacing, no_anchors_x, no_anchors_y, height):
    # iterations represents the number of slices through the cone volume
    # a higher number will theoretically be more accurate for a higher computation speed
    # the areas arent accurate already though because areas are calculated based on 64 sided polygons
    # therefore 100% accuracy will never be guarenteed
    ITERATIONS = 100

    # list of areas, where area is a function of x, and x is a function of the iteration number.
    areas = []

    for n in range(0,ITERATIONS+1):
        diameter = (n / ITERATIONS) * height
        areas.append(area_circle_group(x_spacing, y_spacing, no_anchors_x, no_anchors_y, diameter))

    # might need to add a weight of 0.5 for the start and ending point of the iteration,     
    return sum(areas) / len(areas) * height

volume_group_anchors(x_spacing, y_spacing, no_anchors_x, no_anchors_y, height)

839.5967776666477

In [44]:
# THIS CELL CREATES A PLOT OF THE ARRANGEMENT

# import pythreejs
from pythreejs import *

# create a list of cones that are to be added to the scene
cones = []
for x in range(0,no_anchors_x):
        for y in range(0, no_anchors_y):
            # for each x and y coordinate add a cone. 
            cones.append(
                Mesh(geometry=ConeGeometry(radius=height/2, radialSegments=64, height=height), 
                        material=MeshLambertMaterial(color='blue',transparent=True, opacity=0.5, depthTest=False),
                        position=[x*x_spacing, 0, y*y_spacing])
            )

c = PerspectiveCamera(position=[10, 15, 10], up=[0, 1, 0],
                      children=[DirectionalLight(color='white', position=[3, 5, 1], intensity=0.5)])

scene = Scene(children=cones+[c, AmbientLight(color='#777777')])

renderer = Renderer(camera=c, 
                    scene=scene, 
                    controls=[OrbitControls(controlling=c)])

display(renderer)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.5, position=(3.0, 5.0,…